In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from delta import *

# URL del Master (definida en docker-compose)
master_url = "spark://spark-master:7077"

# Configuración: Añadimos los paquetes JAR de Delta Lake al classpath
builder = SparkSession.builder \
    .appName("Lab_SECOP_Bronze") \
    .master(master_url) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "1g") 

# Iniciar Sesión
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# --- LECTURA ---
# Leemos el CSV mapeado en el volumen. inferSchema es lento pero útil para empezar.
df_raw = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .load("notebooks/data/SECOP_II_Contratos_Electronicos.csv")

# --- ESCRITURA BRONCE ---
# Guardamos en Delta. Esto crea archivos Parquet + _delta_log
df_raw.write.format("delta").mode("overwrite").save("data/lakehouse/bronze/secop")

print(f"Ingesta completada. Registros: {df_raw.count()}")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f3163564-1377-47ad-ab8a-0bf43fe80195;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 159ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/app/notebooks/notebooks/data/SECOP_II_Contratos_Electronicos.csv.

26/02/01 19:30:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
